# Search Health Inluencer Promt | Perplexity | Sonar-pro Model

In [1]:
from openai import OpenAI
import requests
import json
from api_key import API_KEY

In [2]:
YOUR_API_KEY = API_KEY

In [3]:
def true_score( claims, positive):

    total_messages = claims + positive
    
    if claims == 0: 
        trust_score  =  1        
    #elif positive == 0 and claims >=0:
    #    trust_score  =  0
    else: 
        trust_score  = 1 - (claims / total_messages )
    return trust_score

# Test

claims = 3
positive = 0
true_score( claims, positive)

0.0

In [4]:
#influencer_name = "Dr. Sanjay Gupta"
influencer_name = "Dr. Sanjay Gupta (IG: @drsanjaygupta, FB: @SanGuptaMD, Twitter: @drsanjaygupta)"

url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "sonar-pro",
    "messages": [
        {
            "role": "system",
            "content": "You are an expert social media analyst and influencer marketing specialist. Provide detailed, accurate information about influencers in a structured format."
        },
        {
            "role": "user",
            "content": f"""Provide the following information about the influencer {influencer_name} as a JSON object:
            1. subject name without titles or honorifics, only the name name this variable as name
            2. Follower count on major social networks (Instagram, TikTok, YouTube, Twitter, Facebook), name this variable followers
            3. Estimated yearly revenue from social media activities as single figure dollar number, and name this variable as estimatedRevenueYearly
            4. Main content categories or topics as single word, name this variable as categories
            5. Specific products or brands frequently promoted as single element, name this variable as products
            6. Count Significant claims about expertise or notable controversies as number, name this variable as claims
            7. Count Positive feedback for their advice as a number, name this variable as positive
            8. Google Scholar citations, name this variable as citations
            9. Published books names, name this variable as books
            10. His/her social network user name (Instagram, TikTok, YouTube, Twitter, Facebook), name this variable as userNames
            11. Consultation date, name this variable as consultationDate
            12. Make a summary or mini bio with not more than 200 words.  This variable as bio.
            
            
            Format your response as a valid JSON object without any additional text."""
        }
    ]
}

response = requests.post(url, headers=headers, json=payload)
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    try:
        response_data = response.json()
        if 'choices' in response_data and isinstance(response_data['choices'], list) and len(response_data['choices']) > 0:
            content = response_data['choices'][0]['message']['content']
            try:
                parsed_content = json.loads(content)
            
                total_snf = 0
                for snf in parsed_content['followers'].values():
                    if snf:
                        total_snf += snf
                        
                parsed_content['totalSNFollowers'] = total_snf

                parsed_content['trueScore'] = true_score(parsed_content['claims'], parsed_content['positive'])
                
                print("Parsed JSON content:")
                print(json.dumps(parsed_content, indent=2))

                file_name = parsed_content['name'].split(" ")[-1] + ".json"
                
                with open(file_name, 'w') as json_file:
                    json.dump(parsed_content, json_file, indent=4)
                json_file.close()
                
            except json.JSONDecodeError:
                print("The content is not in JSON format. Here's the raw content:")
                print(content)
        else:
            print("Unexpected response structure. Full response:")
            print(json.dumps(response_data, indent=2))
    except Exception as e:
        print(f"Error processing response: {e}")
        print("Full response:")
        print(response.text)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Status Code: 200
Parsed JSON content:
{
  "name": "Sanjay Gupta",
  "followers": {
    "Instagram": 245476,
    "Twitter": 2500000,
    "Facebook": 96700
  },
  "estimatedRevenueYearly": 90000,
  "categories": [
    "Health",
    "Medicine",
    "Neurosurgery",
    "Journalism"
  ],
  "products": "Books",
  "claims": 0,
  "positive": 1414,
  "citations": 21268,
  "books": [
    "Chasing Life",
    "Cheating Death",
    "Monday Mornings: A Novel",
    "Keep Sharp"
  ],
  "userNames": {
    "Instagram": "@drsanjaygupta",
    "Twitter": "@drsanjaygupta",
    "Facebook": "@SanGuptaMD"
  },
  "consultationDate": "2025-01-25",
  "bio": "Dr. Sanjay Gupta is a renowned neurosurgeon, medical reporter, and author. He serves as CNN's chief medical correspondent and has won multiple Emmy Awards for his work. Gupta is an associate professor of neurosurgery at Emory University School of Medicine and associate chief of neurosurgery at Grady Memorial Hospital. He has authored several bestselling books

In [ ]:
parsed_content['claims'] / (parsed_content['claims'] + parsed_content['positive'])

In [5]:
# Total followers test
245476 + 2500000 + 96700

2842176

There is a problem with the Perplexity API results. Sometimes the values of the followers change. It is left for the purposes of the exercise, but it is an aspect to take into account in the future, in case a solution like this is used.

## Perplexity Function

In [6]:
def influencer(influencer_name):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {YOUR_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "sonar-pro",
        "messages": [
            {
                "role": "system",
                "content": "You are an expert social media analyst and influencer marketing specialist. Provide detailed, accurate information about influencers in a structured format."
            },
            {
                "role": "user",
                "content": f"""Provide the following information about the influencer {influencer_name} as a JSON object:
                1. subject name without titles or honorifics, only the name name this variable as name
                2. Follower count on major social networks (Instagram, TikTok, YouTube, Twitter, Facebook), name this variable followers
                3. Estimated yearly revenue from social media activities as single figure dollar number, and name this variable as estimatedRevenueYearly
                4. Main content categories or topics as single word, name this variable as categories
                5. Specific products or brands frequently promoted as single element, name this variable as products
                6. Count Significant claims about expertise or notable controversies as number, name this variable as claims
                7. Count Positive feedback for their advice as a number, name this variable as positive
                8. Google Scholar citations, name this variable as citations
                9. Published books names, name this variable as books
                10. His/her social network user name (Instagram, TikTok, YouTube, Twitter, Facebook), name this variable as userNames
                11. Consultation date, name this variable as consultationDate
                12. Make a summary or mini bio with not more than 200 words.  This variable as bio.
                
                
                Format your response as a valid JSON object without any additional text."""
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=payload)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        try:
            response_data = response.json()
            if 'choices' in response_data and isinstance(response_data['choices'], list) and len(response_data['choices']) > 0:
                content = response_data['choices'][0]['message']['content']
                try:
                    parsed_content = json.loads(content)
                
                    total_snf = 0
                    for snf in parsed_content['followers'].values():
                        if snf:
                            total_snf += snf
                            
                    parsed_content['totalSNFollowers'] = total_snf
    
                    parsed_content['trueScore'] = true_score(parsed_content['claims'], parsed_content['positive'])
                    
                    print("Parsed JSON content:")
                    print(json.dumps(parsed_content, indent=2))
    
                    file_name = parsed_content['name'].split(" ")[-1] + ".json"
                    
                    with open(file_name, 'w') as json_file:
                        json.dump(parsed_content, json_file, indent=4)
                    json_file.close()
                    
                except json.JSONDecodeError:
                    print("The content is not in JSON format. Here's the raw content:")
                    print(content)
            else:
                print("Unexpected response structure. Full response:")
                print(json.dumps(response_data, indent=2))
        except Exception as e:
            print(f"Error processing response: {e}")
            print("Full response:")
            print(response.text)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [7]:
influencer_name = "Kevin Pho, M.D. (IG: @kevinmd, FB: @Kevinmdblog, Twitter: @kevinmd)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Kevin Pho",
  "followers": {
    "Instagram": 9186,
    "Twitter": 250000,
    "Facebook": 250000
  },
  "estimatedRevenueYearly": 500000,
  "categories": [
    "healthcare",
    "medicine",
    "socialmedia"
  ],
  "products": "books",
  "claims": 0,
  "positive": 5,
  "citations": 0,
  "books": [
    "Establishing, Managing, and Protecting Your Online Reputation: A Social Media Guide for Physicians and Medical Practices"
  ],
  "userNames": {
    "Instagram": "@kevinphomd",
    "Twitter": "@kevinmd",
    "Facebook": "@Kevinmdblog"
  },
  "consultationDate": "2025-01-26",
  "bio": "Kevin Pho is a board-certified internal medicine physician, national media commentator, and social media leader in healthcare. He founded KevinMD.com in 2004, which has become a leading platform for physicians, advanced practitioners, nurses, medical students, and patients to share insights and stories. Pho is also an acclaimed keynote speaker, addressing m

In [8]:
influencer_name = "Danielle Ofri, M.D., Ph.D. (IG: @danielle_ofri, FB: @danielleOfriMD, Twitter: @danielleofri)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Danielle Ofri",
  "followers": {
    "Instagram": 1468,
    "Twitter": 12000,
    "Facebook": 5000
  },
  "estimatedRevenueYearly": 50000,
  "categories": [
    "Medicine",
    "Writing",
    "Healthcare"
  ],
  "products": "Books",
  "claims": 0,
  "positive": 500,
  "citations": 1000,
  "books": [
    "When We Do Harm: A Doctor Confronts Medical Error",
    "What Patients Say, What Doctors Hear",
    "What Doctors Feel",
    "Medicine in Translation",
    "Singular Intimacies",
    "Incidental Findings"
  ],
  "userNames": {
    "Instagram": "@danielle_ofri",
    "Twitter": "@danielleofri",
    "Facebook": "@danielleOfriMD"
  },
  "consultationDate": "2024-01-26",
  "bio": "Dr. Danielle Ofri is a practicing physician at Bellevue Hospital and a clinical professor of medicine at NYU. She is a prolific writer, contributing to publications like the New York Times and The New Yorker. As the founder and editor-in-chief of the Bellevue Lite

In [9]:
influencer_name = "Sarah Kliff (FB: @sarahkliff, Twitter: @sarahkliff)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Sarah Kliff",
  "followers": {
    "Twitter": 154000,
    "Facebook": 0
  },
  "estimatedRevenueYearly": 150000,
  "categories": [
    "healthcare",
    "policy",
    "journalism"
  ],
  "products": "None",
  "claims": 0,
  "positive": 0,
  "citations": 0,
  "books": [],
  "userNames": {
    "Twitter": "@sarahkliff",
    "Facebook": "@skliff"
  },
  "consultationDate": "2025-01-26",
  "bio": "Sarah Kliff is a prominent health policy journalist and investigative reporter for The New York Times. With nearly a decade of experience covering healthcare issues, she has become one of the country's leading voices on the Affordable Care Act and other health policy matters. Kliff's reporting has influenced legislation, been cited by the Supreme Court, and led to changes in hospital billing practices. Previously, she worked at Vox and The Washington Post, where she gained recognition for making complex policy issues understandable to the public. 

In [10]:
influencer_name = "Dr. Mike Varshavski (IG: @doctor.mike, FB: @realdoctormike, Twitter: @realdoctormike)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Mike Varshavski",
  "followers": {
    "Instagram": 4497931,
    "YouTube": 13200000,
    "TikTok": 11400000,
    "Twitter": 2100000,
    "Facebook": 4200000
  },
  "estimatedRevenueYearly": 5000000,
  "categories": [
    "Health",
    "Medicine",
    "Lifestyle"
  ],
  "products": "Skincare",
  "claims": 0,
  "positive": 1000000,
  "citations": 0,
  "books": [],
  "userNames": {
    "Instagram": "doctor.mike",
    "YouTube": "doctormike",
    "TikTok": "doctor.mike",
    "Twitter": "realdoctormike",
    "Facebook": "realdoctormike"
  },
  "consultationDate": "2025-01-26",
  "bio": "Dr. Mike Varshavski is a board-certified family medicine physician and social media influencer known as 'Doctor Mike'. With over 23 million followers across platforms, he makes medical information accessible and engaging. Born in Russia, he immigrated to the US at age 6 and became a doctor by 24. Dr. Mike uses his platform to combat health misinformation, p

In [11]:
influencer_name = "Dr. Jennifer Gunter (IG: @drjengunter, FB: @DrJGunter, Twitter: @DrJenGunter)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Jennifer Gunter",
  "followers": {
    "Instagram": 200000,
    "Twitter": 360000,
    "Facebook": 50000
  },
  "estimatedRevenueYearly": 500000,
  "categories": [
    "women's health",
    "gynecology",
    "medical misinformation"
  ],
  "products": "books",
  "claims": 5,
  "positive": 10000,
  "citations": 50,
  "books": [
    "The Preemie Primer",
    "The Vagina Bible",
    "The Menopause Manifesto"
  ],
  "userNames": {
    "Instagram": "@drjengunter",
    "Twitter": "@DrJenGunter",
    "Facebook": "@DrJGunter"
  },
  "consultationDate": "2025-01-26",
  "bio": "Dr. Jennifer Gunter is a renowned OB-GYN, pain medicine physician, and women's health advocate. Known as 'Twitter's resident gynecologist,' she has gained prominence for debunking medical misinformation and promoting evidence-based women's health information. Gunter is a New York Times bestselling author, with books like 'The Vagina Bible' and 'The Menopause Manifesto.' S

In [12]:
influencer_name = "Dr. David Katz (IG: @davidkatz, FB: @DrDavidKatz, Twitter: @DrDavidKatz)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "David Katz",
  "followers": {
    "Instagram": 8259,
    "Twitter": 54000,
    "Facebook": 28000
  },
  "estimatedRevenueYearly": 150000,
  "categories": [
    "health",
    "nutrition",
    "medicine",
    "lifestyle"
  ],
  "products": "Diet ID",
  "claims": 5,
  "positive": 1000,
  "citations": 200,
  "books": [
    "How to Eat",
    "The Truth About Food",
    "Disease-Proof",
    "The Flavor Point Diet",
    "The Way to Eat"
  ],
  "userNames": {
    "Instagram": "@davidkatz",
    "Twitter": "@DrDavidKatz",
    "Facebook": "@DrDavidKatz"
  },
  "consultationDate": "2025-01-26",
  "bio": "Dr. David Katz is a renowned expert in preventive medicine, nutrition, and lifestyle medicine. He is the founding director of Yale University's Yale-Griffin Prevention Research Center, past president of the American College of Lifestyle Medicine, and founder of the True Health Initiative. Katz has authored numerous books, holds multiple patents, a

In [13]:
influencer_name = "Regina Holliday (IG: @regina_holliday, Twitter: @ReginaHolliday)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Regina Holliday",
  "followers": {
    "Twitter": 20000,
    "Facebook": 3000
  },
  "estimatedRevenueYearly": 50000,
  "categories": [
    "healthcare",
    "patientrights",
    "art"
  ],
  "products": "patientadvocacyart",
  "claims": 0,
  "positive": 200,
  "citations": 5,
  "books": [
    "The Walking Wall: 73 Cents to the Walking Gallery",
    "The Writing on the Wall"
  ],
  "userNames": {
    "Twitter": "@ReginaHolliday",
    "Instagram": "@regina_holliday"
  },
  "consultationDate": "2025-01-26",
  "bio": "Regina Holliday is a patient rights activist, artist, and founder of the Walking Gallery movement. After losing her husband to kidney cancer in 2009, she began using art to advocate for transparency in medical records and patient rights. Holliday paints murals and jacket backs to tell patient stories and has spoken at numerous healthcare conferences. Her work has been featured in major media outlets and has influenced health

In [14]:
influencer_name = "Angel Love Davis (IG: @iloveangel2, Twitter: @Iloveangel2, Youtube: @angellovedavis)"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Angel Love Davis",
  "followers": {
    "Instagram": 1000000,
    "YouTube": 1200,
    "Twitter": 14000,
    "Facebook": 500000
  },
  "estimatedRevenueYearly": 200000,
  "categories": [
    "lifestyle",
    "fashion",
    "nursing",
    "entrepreneurship"
  ],
  "products": "Life Angels scrubs",
  "claims": 0,
  "positive": 5000,
  "citations": 0,
  "books": [],
  "userNames": {
    "Instagram": "iloveangel2",
    "Twitter": "Iloveangel2",
    "YouTube": "angellovedavis"
  },
  "consultationDate": "2025-01-26",
  "bio": "Angel Love Davis is a multi-talented individual who has made a name for herself as a registered nurse, reality TV star, actress, and entrepreneur. Rising to fame on VH1's 'Basketball Wives LA', she has since leveraged her popularity to build a successful career in entertainment and fashion. Angel holds a nursing degree and has founded Life Angels, a stylish scrubs line for healthcare professionals. With over 1 million

In [15]:
influencer_name = "Michelle Lewin"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Michelle Lewin",
  "followers": {
    "Instagram": 16000000,
    "YouTube": 402000,
    "Twitter": 457000,
    "Facebook": 8600000
  },
  "estimatedRevenueYearly": 1200000,
  "categories": [
    "Fitness",
    "Lifestyle",
    "Modeling"
  ],
  "products": "FitPlan",
  "claims": 0,
  "positive": 1000,
  "citations": 0,
  "books": [],
  "userNames": {
    "Instagram": "@michelle_lewin",
    "YouTube": "Michelle Lewin",
    "Twitter": "@Michelle_Lewin",
    "Facebook": "MichelleLewin"
  },
  "consultationDate": "2025-01-26",
  "bio": "Michelle Lewin is a Venezuelan fitness influencer, model, and entrepreneur who has risen to prominence in the fitness industry. With over 16 million Instagram followers, she is known for her impressive physique and dedication to fitness. Lewin shares workout routines, nutritional tips, and lifestyle content across various social media platforms. She has transitioned from a catwalk model to a fitness model a

In [16]:
influencer_name = "Deepak Chopra"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Deepak Chopra",
  "followers": {
    "Instagram": 2736378,
    "Facebook": 3000000,
    "Twitter": 3000000,
    "YouTube": 500000
  },
  "estimatedRevenueYearly": 5000000,
  "categories": [
    "Wellness",
    "Spirituality",
    "Meditation",
    "Ayurveda",
    "Self-help"
  ],
  "products": [
    "Books",
    "Meditation apps",
    "Wellness retreats",
    "Ayurvedic products"
  ],
  "claims": 10,
  "positive": 1000000,
  "citations": 5849,
  "books": [
    "The Seven Spiritual Laws of Success",
    "Ageless Body, Timeless Mind",
    "Perfect Health",
    "Quantum Healing",
    "The Future of God",
    "Metahuman",
    "Total Meditation"
  ],
  "userNames": {
    "Instagram": "@deepakchopra",
    "Facebook": "DeepakChopra",
    "Twitter": "@DeepakChopra",
    "YouTube": "@DeepakChopra"
  },
  "consultationDate": "2025-01-26",
  "bio": "Deepak Chopra is a world-renowned figure in alternative medicine and personal transformation. Born

In [17]:
influencer_name = "Kait"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Kaitlyn Reichert",
  "followers": {
    "Instagram": 2253977,
    "TikTok": 347000,
    "YouTube": null,
    "Twitter": null,
    "Facebook": null
  },
  "estimatedRevenueYearly": 144000,
  "categories": [
    "fashion",
    "lifestyle",
    "beauty",
    "wedding"
  ],
  "products": "PAINT Nail Bar St. Louis",
  "claims": 0,
  "positive": 0,
  "citations": 0,
  "books": [],
  "userNames": {
    "Instagram": "tru.kait",
    "TikTok": null,
    "YouTube": null,
    "Twitter": null,
    "Facebook": null
  },
  "consultationDate": "2025-01-26",
  "bio": "Kaitlyn Reichert, known as Kait, is a social media influencer based in St. Louis, Missouri. She shares content about fashion, lifestyle, beauty, and her wedding journey. With over 2 million Instagram followers, Kait engages her audience with style inspiration and trendy finds. Her content strategy blends professional expertise with authentic storytelling, resonating with executives, entre

In [18]:
influencer_name = "Kait Malthaner"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Kait Malthaner",
  "followers": {
    "Instagram": 154000,
    "YouTube": 579000,
    "Facebook": null,
    "Twitter": null,
    "TikTok": null
  },
  "estimatedRevenueYearly": 150000,
  "categories": [
    "nutrition",
    "health",
    "keto",
    "carnivore"
  ],
  "products": [
    "Berberine supplements",
    "Inositol supplements",
    "Collagen powder"
  ],
  "claims": 2,
  "positive": 500,
  "citations": 0,
  "books": [],
  "userNames": {
    "Instagram": "healthcoachkait",
    "YouTube": "Kait Malthaner (BSc Nutrition & Exercise)",
    "Facebook": "healthcoachkait",
    "Twitter": "healthcoachkait"
  },
  "consultationDate": "2025-01-26",
  "bio": "Kait Malthaner is a certified health coach and YouTuber based in Australia, specializing in ketogenic and carnivore diets. With a BSc in Nutrition & Exercise, she focuses on helping clients achieve weight loss and fitness goals through a nutrient-dense, high-fat, low-carb approach. 

In [19]:
influencer_name = "Dr. Carlos Jaramillo"
influencer(influencer_name)

Status Code: 200
Parsed JSON content:
{
  "name": "Carlos Jaramillo",
  "followers": {
    "Instagram": 2233313,
    "YouTube": 100000
  },
  "estimatedRevenueYearly": 500000,
  "categories": [
    "health",
    "nutrition",
    "functionalmedicine"
  ],
  "products": "supplements",
  "claims": 1,
  "positive": 100000,
  "citations": 19519,
  "books": [
    "El milagro metab\u00f3lico"
  ],
  "userNames": {
    "Instagram": "drcarlosjaramillo",
    "YouTube": "DrCarlos"
  },
  "consultationDate": "2025-01-26",
  "bio": "Dr. Carlos Jaramillo is a Colombian functional medicine specialist and nutrition expert with a significant social media presence. He studied at La Sabana University and the Institute of Functional Medicine in the United States. Jaramillo focuses on lifestyle-based health strategies and educates his audience on metabolism, nutrition, and overall well-being through various platforms including Instagram, YouTube, books, and virtual courses. His content aims to empower indi

In above case, youtube followers are more than 5 millions.  